# GamessFortranReader tutorial

`GamessFortranReader` is a class containing methods for reading Gamess sequential, unformatted fortran files containig

* two-electron integerals over AO's (\$JOB.F08)
* two-electron integrals over MO's (\$JOB.F09)
* two-particle density matrix (\$JOB.F15)

# IMPORTANT NOTE

As with all python iterable objects the counting starts from 0 not from 1 as in fortran. This means for example that the first element of the two electron integral supermatix **W** will be $<00|00>$ in python and not $<11|11>$ as in fortran. This of course means that the orbitals will be also indexed from 0 and all the loop over the elements of matrices and vectors should start from 0.

In [3]:
from chemtools.gamessreader import GamessFortranReader, print_twoe, ijkl, factor
from chemtools.gamessus import Gamess
import numpy as np
import os

First we need to generate some files to work on. Below I'm using my own gamess installation with custom `rungamessave` script which compies the .F?? (with integrals etc.) files to my current directory after the job is finished. 

In [4]:
gamess = Gamess(executable="/home/lmentel/Programs/gamess-us-dec2014/rungmssave",
                version="00",
                runopts=None,
                scratch="/home/lmentel/scratch")

As an example I will use H$_2$ from earlier. Here's the input

In [6]:
inpstr = """ $CONTRL scftyp=rhf runtyp=energy maxit=30 mult=1 ispher=1
     itol=30 icut=30 units=bohr cityp=guga qmttol=1.0e-8 $END
 $SYSTEM timlim=525600 mwords=100 $END
 $SCF dirscf=.false. $END
 $CIINP
    nrnfg(6)=1
 $END
 $CIDRT iexcit=2 nfzc=0 ndoc=1 nval=27 group=d2h stsym=ag
        mxnint=14307305 $END
 $GUGDIA prttol=1.0e-6 cvgtol=1.0e-10 $END
 $DATA
H2 cc-pVTZ
dnh 2

H    1.00       0.000000       0.000000       0.700000
S   3
  1     33.8700000              0.0060680        
  2      5.0950000              0.0453080        
  3      1.1590000              0.2028220        
S   1
  1      0.3258000              1.0000000        
S   1
  1      0.1027000              1.0000000        
P   1
  1      1.4070000              1.0000000        
P   1
  1      0.3880000              1.0000000        
D   1
  1      1.0570000              1.0000000        

 $END
"""

I'll create a `temp` folder as my workdir for the calculation

In [7]:
os.mkdir('temp')
os.chdir('temp')
os.getcwd()

'/home/lmentel/Devel/chemtools/examples/ipython_notebooks/temp'

Now I can write the input file

In [8]:
inpfile = "h2_eq_pvtz_fci.inp"
with open(inpfile, 'w') as inp:
    inp.write(inpstr)

and run the calculation

In [9]:
logfile = gamess.run(inpfile)

In [10]:
!ls

h2_eq_pvtz_fci.F05  h2_eq_pvtz_fci.F10	h2_eq_pvtz_fci.F14  h2_eq_pvtz_fci.inp
h2_eq_pvtz_fci.F08  h2_eq_pvtz_fci.F11	h2_eq_pvtz_fci.F15  h2_eq_pvtz_fci.log
h2_eq_pvtz_fci.F09  h2_eq_pvtz_fci.F12	h2_eq_pvtz_fci.F16


All the important files are there. Now I can create the `GamessFortranReader` giving it the `logfile` (name out the output) as an argument since it need to figure out how many AO's and MO's there are before reading the integral files.

In [11]:
gfr = GamessFortranReader(logfile) 

## reading the two-electron integrals in AO

In [12]:
twoeAO = gfr.read_twoeao()
twoeAO

array([ 0.        ,  0.        ,  0.        , ...,  0.        ,
        0.        ,  0.19766454])

In [13]:
nao = gfr.gp.get_number_of_aos()
print_twoe(twoeAO, nao)

 15  0 15  0          0.04521671131113
 15 15  0  0          0.17480185385836
 15 15 15  0          0.29002632794006
 15 15 15 15          0.34828542821928
 16  0  0  0          0.48007780587717
 16  0 15  0          0.24081849512275
 16  0 15  1          0.15294987166635
 16  0 15 15          0.33802872713839
 16  0 16  0          0.18318350853723
 16  1 15  0          0.15834708632932
 16  1 15 15          0.54967855195145
 16  1 16  0          0.46163404996668
 16  1 16  1          0.17004898534820
 16 15  0  0          0.48066035988211
 16 15  1  0          0.32852136888746
 16 15 15  0          0.36687122957197
 16 15 15  1          0.59864163621333
 16 15 15 15          0.82291439094856
 16 15 16  0          0.45218748830891
 16 15 16  1          0.73291274511781
 16 15 16 15          0.50959740421333
 16 16  0  0          0.30380932605259
 16 16  1  0          0.41471247441065
 16 16  1  1          0.13242281651343
 16 16 15  0          0.20113835226587
 16 16 15  1          0.3

## reading the two-electron integrals in MO

In [14]:
twoeMO = gfr.read_twoemo()
twoeMO

array([ 0.65839032,  0.        ,  0.04145531, ...,  0.        ,
        0.        ,  0.97963496])

In [15]:
nmo = gfr.gp.get_number_of_mos()
print_twoe(twoeMO, nmo)

  0  0  0  0          0.65839031524498
  1  0  1  0          0.04145530744621
  1  1  0  0          0.34153368206470
  1  1  1  1          0.29503960619379
  2  0  0  0          0.13802415602617
  2  0  1  1          0.02444892035465
  2  0  2  0          0.05576222390046
  2  1  1  0         -0.02109259848950
  2  1  2  1          0.03805581831713
  2  2  0  0          0.36427568668547
  2  2  1  1          0.27005757987944
  2  2  2  0          0.03544390622815
  2  2  2  2          0.28088282164510
  3  0  1  0          0.05725581613601
  3  0  2  1         -0.00793046935555
  3  0  3  0          0.09671465761720
  3  1  0  0          0.13322027989492
  3  1  1  1          0.04298419383441
  3  1  2  0          0.03747714852960
  3  1  2  2          0.04478345632012
  3  1  3  1          0.05260248556585
  3  2  1  0          0.02696818525998
  3  2  2  1         -0.01683201350793
  3  2  3  0          0.03716166047742
  3  2  3  2          0.02390028364654
  3  3  0  0          0.4

## reading the two particle density matrix

In [16]:
twoRDM = gfr.read_rdm2()
twoRDM

array([  1.96417238e+00,   0.00000000e+00,  -3.91054292e-02, ...,
         0.00000000e+00,   0.00000000e+00,   1.66895798e-05])

In [17]:
print_twoe(twoRDM, nmo)

  0  0  0  0          1.96417238210484
  1  0  1  0         -0.03910542919187
  1  1  1  1          0.00311425739658
  2  0  0  0          0.01311745043666
  2  0  2  0         -0.03781291501406
  2  1  1  0         -0.00026116013742
  2  1  2  1          0.00150740654098
  2  2  0  0          0.00008760305741
  2  2  2  0         -0.00050564156941
  2  2  2  2          0.00291854421833
  3  0  1  0         -0.04387595693736
  3  0  2  1         -0.00029301943950
  3  0  3  0         -0.05535557340956
  3  1  1  1          0.00349417015099
  3  1  3  1          0.00416440338004
  3  2  1  0         -0.00029301943950
  3  2  2  1          0.00169129724046
  3  2  3  0         -0.00036968445194
  3  2  3  2          0.00213380482357
  3  3  1  1          0.00392042901063
  3  3  3  1          0.00494616211345
  3  3  3  3          0.00624026594696
  4  0  4  0         -0.04146788344151
  4  1  4  1          0.00165119863657
  4  2  4  0         -0.00027693745758
  4  2  4  2          0.0

## Calculate the two-electron energy from the two-electron integrals and 2RDM

In [21]:
energy = 0.0

# loop over elements and sum them
ij = 0
for i in xrange(nmo):
    for j in xrange(i+1):
        ij += 1
        kl = 0
        for k in xrange(nmo):
            for l in xrange(k+1):
                kl += 1
                if ij >= kl:
                    energy += factor(i,j,k,l)*twoeMO[ijkl(i,j,k,l)]*twoRDM[ijkl(i,j,k,l)]
                    #print(i,j,k,l,factor(i,j,k,l), twoeint[ijkl(i,j,k,l)], twordm[ijkl(i,j,k,l)])

print("Two-electron energy: ", 0.5*energy)

('Two-electron energy: ', 0.58866020149518394)


In [20]:
gfr.gp.get_energy_components('ci')

{'ELECTRON-ELECTRON POTENTIAL ENERGY': 0.5886602014,
 'NUCLEAR REPULSION ENERGY': 0.7142857143,
 'NUCLEUS-ELECTRON POTENTIAL ENERGY': -3.6466910029,
 'NUCLEUS-NUCLEUS POTENTIAL ENERGY': 0.7142857143,
 'ONE ELECTRON ENERGY': -2.4752805092,
 'TOTAL ENERGY': -1.1723345936,
 'TOTAL KINETIC ENERGY': 1.1714104937,
 'TOTAL POTENTIAL ENERGY': -2.3437450872,
 'TWO ELECTRON ENERGY': 0.5886602014,
 'VIRIAL RATIO (V/T)': 2.0007888779,
 'WAVEFUNCTION NORMALIZATION': 1.0}

As you can see the energy obtained by contracting twoMO with twoRDM ('Two-electron energy: ', 0.58866020149518394) is the same as the one parsed from the logfile ('ELECTRON-ELECTRON POTENTIAL ENERGY': 0.5886602014,).